In [ ]:
import numpy as np 
import math 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.integrate as integrate
from scipy.optimize import fsolve

In [ ]:
data_1 = {
    'Maturity': [0, 1, 3, 5, 7, 10],
    'CDS_rate': [np.nan, 100, 110, 120, 120, 125]
}
cds_rate = pd.DataFrame(data_1)
cds_rate

# Question 1

In [ ]:
def average_hazard_rate(cds_rate):
    return cds_rate / LGD / 10000

In [ ]:
def survival_prob(maturity, average_hazard_rate):
    return np.exp(-average_hazard_rate * maturity)

In [ ]:
LGD = 0.4
h_rate = []

for i in range(cds_rate.shape[0]):
    maturity = cds_rate['Maturity'].iloc[i] 
    cds_rate_value = cds_rate['CDS_rate'].iloc[i]  
    average_hazard_rate_value = average_hazard_rate(cds_rate_value)
    h_rate.append({'Maturity': maturity, 'Average lambda': average_hazard_rate_value})

h_rate_df = pd.DataFrame(h_rate)
h_rate_df

In [ ]:
h_rate_df['Q(t>T)'] = survival_prob(h_rate_df['Maturity'], h_rate_df['Average lambda'])
h_rate_df.iloc[0, 2] = 1

In [ ]:
h_rate_df['fwd_h_rate'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'fwd_h_rate'] = -np.log(h_rate_df.loc[i, 'Q(t>T)'] / h_rate_df.loc[i - 1, 'Q(t>T)'])\
    / (h_rate_df.loc[i, 'Maturity'] - h_rate_df.loc[i - 1, 'Maturity'])       

In [ ]:
h_rate_df['default_prob'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'default_prob'] = h_rate_df.loc[i - 1, 'Q(t>T)'] - h_rate_df.loc[i, 'Q(t>T)']


In [ ]:
final_1_df = h_rate_df[1:]
final_1_df

# Question 2

### Lambda 1

In [ ]:
def get_lambda_1(r, rate, T_start, T_stop, LGD, payment):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment):
        # Define the payment times
        payment_times = np.linspace(T_start, T_stop, int((T_stop - T_start) * payment) + 1)
    
        # Premium if not default
        init_premium = np.zeros(len(payment_times))
        for i, u in enumerate(payment_times):
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r*u) * (1/payment) * np.exp(-h*u)
        final_premium = np.sum(init_premium)
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                init_acc_premium[i] = rate * np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * ((1/payment)/2) * \
                (np.exp(-h*payment_times[i-1]) - np.exp(-h*payment_times[i]))
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_protection[i] = 0
            else:
                init_protection[i] = np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * \
                (np.exp(-h*payment_times[i-1]) - np.exp(-h*payment_times[i]))
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate/LGD/10000, args=(r, rate, T_start, T_stop, LGD, payment))
    
    return cumulative_lambda


In [ ]:
r = 0.03
rate = 100 / 10000
T_start = 0
T_stop = 1
LGD = 0.4
payment = 4
h_1 = get_lambda_1(r, rate, T_start, T_stop, LGD, payment)[0]
h_1

### Lambda 2

In [ ]:
def get_lambda_2(r, rate, T_start, T_stop, LGD, payment, h_1):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1):
        
        # Define the payment times
        payment_times = np.linspace(T_start, T_stop, int((T_stop - T_start) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if payment_times[i] <= 1:
                h_func = -h_1*payment_times[i]    
            else:
                h_func = -h*payment_times[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r*payment_times[i]) * (1/payment) * np.exp(h_func)      
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                if payment_times[i] <= 1:
                    h_func_1 = -h_1*payment_times[i-1]
                    h_func_2 = -h_1*payment_times[i]    
                else:
                    h_func_1 = -h*payment_times[i-1]
                    h_func_2 = -h*payment_times[i]   
                init_acc_premium[i] = rate * np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * ((1/payment)/2) * \
                (np.exp(h_func_1) - np.exp(h_func_2))     
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_protection[i] = 0
            else:
                if payment_times[i] <= 1:
                    h_func_1 = -h_1*payment_times[i-1]
                    h_func_2 = -h_1*payment_times[i]
                else:
                    h_func_1 = -h*payment_times[i-1]
                    h_func_2 = -h*payment_times[i]
                init_protection[i] = np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * \
                (np.exp(h_func_1) - np.exp(h_func_2))
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1))
    
    return cumulative_lambda


In [ ]:
r = 0.03
rate_3 = 110 / 10000
T_start = 0
T_stop = 3
LGD = 0.4
payment = 4
h_2 = get_lambda_2(r, rate_3, T_start, T_stop, LGD, payment, h_1)[0]

### Lambda 3

In [ ]:
def get_lambda_3(r, rate, T_start, T_stop, LGD, payment, h_1, h_2):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, T_stop, int((T_stop - 3) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            h_func = -h * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h * payment_times_3[i-1]
                h_func_2 = -h * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h * payment_times_3[i-1]
                h_func_2 = -h * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2))
    
    return cumulative_lambda

In [ ]:
r = 0.03
rate_3 = 120 / 10000
T_start = 0
T_stop = 5
LGD = 0.4
payment = 4
h_3 = get_lambda_3(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2)[0]

### Lambda 4

In [ ]:
def get_lambda_4(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, 5, int(2 * payment) + 1)
        payment_times_4 = np.linspace(5, T_stop, int((T_stop - 5) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            h_func = -h_3 * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            h_func = -h * payment_times_4[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * payment_times_4[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h * payment_times_4[i-1]
                h_func_2 = -h * payment_times_4[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h * payment_times_4[i-1]
                h_func_2 = -h * payment_times_4[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3))
    
    return cumulative_lambda


In [ ]:
r = 0.03
rate_3 = 120 / 10000
T_start = 0
T_stop = 7
LGD = 0.4
payment = 4
h_4 = get_lambda_4(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2, h_3)[0]

### Lambda 5

In [ ]:
def get_lambda_5(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, 5, int(2 * payment) + 1)
        payment_times_4 = np.linspace(5, 7, int(2 * payment) + 1)
        payment_times_5 = np.linspace(7, T_stop, int((T_stop - 7) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            h_func = -h_3 * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            h_func = -h_4 * payment_times_4[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * payment_times_4[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            h_func = -h * payment_times_5[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = rate * np.exp(-r * payment_times_5[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h_4 * payment_times_4[i-1]
                h_func_2 = -h_4 * payment_times_4[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                h_func_1 = -h * payment_times_5[i-1]
                h_func_2 = -h * payment_times_5[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = rate * np.exp(-r * (payment_times_5[i] + payment_times_5[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h_4 * payment_times_4[i-1]
                h_func_2 = -h_4 * payment_times_4[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                h_func_1 = -h * payment_times_5[i-1]
                h_func_2 = -h * payment_times_5[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = np.exp(-r * (payment_times_5[i] + payment_times_5[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4))
    
    return cumulative_lambda


In [ ]:
r = 0.03
rate_3 = 125 / 10000
T_start = 0
T_stop = 10
LGD = 0.4
payment = 4
h_5 = get_lambda_5(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4)[0]

In [ ]:
h_list = [0, h_1, h_2, h_3, h_4, h_5]

In [ ]:
m_list = h_rate_df['Maturity'].values

In [ ]:
distances = [m_list[i+1] - m_list[i] for i in range(len(m_list)-1)]

In [ ]:
ah_list = np.zeros(len(h_list))
ah_list

In [ ]:
ah_list[1] = h_list[1] * distances[0]

In [ ]:
ah_list[2] = (h_list[2] * distances[1] + h_list[1] * distances[0]) / np.sum(distances[0:2])

In [ ]:
ah_list[3] = (h_list[3] * distances[2] + h_list[2] * distances[1] + h_list[1] * distances[0]) / np.sum(distances[0:3])

In [ ]:
ah_list[4] = (h_list[4] * distances[3] + h_list[3] * distances[2] + h_list[2] * distances[1] + h_list[1] * distances[0]) / np.sum(distances[0:4])

In [ ]:
ah_list[5] = (h_list[5] * distances[4] + h_list[4] * distances[3] + h_list[3] * distances[2] + h_list[2] * distances[1] + h_list[1] * distances[0]) / np.sum(distances[0:5])

In [ ]:
ah_list

In [ ]:
data_2_df = pd.DataFrame(index=h_rate_df.index, columns=h_rate_df.columns)
data_2_df['Maturity'] = h_rate_df['Maturity']
data_2_df['Average lambda'] = ah_list

In [ ]:
data_2_df['Q(t>T)'] = survival_prob(data_2_df['Maturity'], data_2_df['Average lambda'])
data_2_df.iloc[0, 2] = 1

In [ ]:
for i in range(1, data_2_df.shape[0]):
    data_2_df.loc[i, 'fwd_h_rate'] = -np.log(data_2_df.loc[i, 'Q(t>T)'] / data_2_df.loc[i - 1, 'Q(t>T)'])\
    / (data_2_df.loc[i, 'Maturity'] - data_2_df.loc[i - 1, 'Maturity'])   

In [ ]:
for i in range(1, data_2_df.shape[0]):
    data_2_df.loc[i, 'default_prob'] = data_2_df.loc[i - 1, 'Q(t>T)'] - data_2_df.loc[i, 'Q(t>T)']

In [ ]:
final_2_df = data_2_df[1:]
final_2_df

In [ ]:
final_1_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


maturity_1 = [1, 3, 5, 7, 10]
average_lambda_1 = final_1_df['Average lambda']
default_prob_1 = final_1_df['default_prob']


maturity_2 = [1, 3, 5, 7, 10]
average_lambda_2 = final_2_df['Average lambda']
default_prob_2 = final_2_df['default_prob']


bar_width = 0.35
index = np.arange(len(maturity_1))


fig, ax = plt.subplots(figsize=(10, 6))
bar1 = ax.bar(index - bar_width/2, default_prob_1, bar_width, label='Default Probability (Final 1)')
bar2 = ax.bar(index + bar_width/2, average_lambda_1, bar_width, label='Average Lambda (Final 1)')

ax.set_xlabel('Maturity')
ax.set_ylabel('Values')
ax.set_title('Question 1: Default Probability and Average Lambda vs. Maturity')
ax.set_xticks(index)
ax.set_xticklabels(maturity_1)
ax.legend()


fig, ax = plt.subplots(figsize=(10, 6))
bar3 = ax.bar(index - bar_width/2, default_prob_2, bar_width, label='Default Probability (Final 2)')
bar4 = ax.bar(index + bar_width/2, average_lambda_2, bar_width, label='Average Lambda (Final 2)')

ax.set_xlabel('Maturity')
ax.set_ylabel('Values')
ax.set_title('Question 2: Default Probability and Average Lambda vs. Maturity')
ax.set_xticks(index)
ax.set_xticklabels(maturity_2)
ax.legend()


plt.show()
